In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [2]:
df = pd.read_excel(r"D:\Jupyter\Data Science\ML\SUPERVISIED LEARNING\CLASSIFICATION\DIABETIC DATASET\cleandiabetic.xlsx")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,89,66,23,4.546836,28.1,0.408656,2.140695,0
1,0,137,40,35,5.517848,43.1,1.512614,2.396782,1
2,3,78,50,32,4.447960,31.0,0.497996,2.258101,1
3,2,197,70,45,8.158305,30.5,0.397492,2.698168,1
4,1,189,60,23,9.457800,30.1,0.630872,2.771488,1


In [3]:
X = df.drop('Outcome',axis=1)
y = df['Outcome']

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=True)

# WITH PCA

In [6]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=0.95)
X_train_pca = pca_model.fit_transform(X_train)
X_test_pca =pca_model.transform(X_test)

In [7]:
pca_model.explained_variance_ratio_

array([0.74566143, 0.13917885, 0.09059457])

# LOGISTIC REGREESION WITH PCA

In [8]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()
log_reg.fit(X_train_pca,y_train)

ypred_train = log_reg.predict(X_train_pca)
ypred_test = log_reg.predict(X_test_pca)
# train accuarcy
from sklearn.metrics import accuracy_score
print('train accuracy',accuracy_score(y_train,ypred_train))
# test accuracy

print('testaccuracy',accuracy_score(y_test,ypred_test))


train accuracy 0.7859424920127795
testaccuracy 0.759493670886076


# KNN WITH PCA

In [9]:
#modelling
from sklearn.neighbors import KNeighborsClassifier
knn_pca_model = KNeighborsClassifier()
knn_pca_model.fit(X_train_pca,y_train)

#prediction 
y_pred_test = knn_pca_model.predict(X_test_pca)
y_pred_train = knn_pca_model.predict(X_train_pca)

#Evalution 
from sklearn.metrics import accuracy_score
print('train accuracy',accuracy_score(y_train,y_pred_train))
print('test accuracy',accuracy_score(y_test,y_pred_test))



train accuracy 0.8274760383386581
test accuracy 0.6962025316455697


# SVM WITH PCA

In [10]:
from sklearn.svm import SVC
model = SVC(kernel = 'linear',C =10)
model.fit(X_train_pca,y_train)
y_pred_train = model.predict(X_train_pca)
y_pred_test = model.predict(X_test_pca)

print('train:',accuracy_score(y_train,ypred_train))
print("test:",accuracy_score(y_test,ypred_test))

train: 0.7859424920127795
test: 0.759493670886076


# DECSION TREE WITH PCA

In [11]:
from sklearn.tree import DecisionTreeClassifier
model= DecisionTreeClassifier(criterion='gini',max_depth= 3,random_state=True)
model.fit(X_train_pca,y_train) 

ypred_train = model.predict(X_train_pca)
ybase_pred = model.predict(X_test_pca)

from sklearn.metrics import accuracy_score
print('train accuracy',accuracy_score(ypred_train,y_train))
print('test accuracy',accuracy_score(ybase_pred,y_test))


train accuracy 0.8115015974440895
test accuracy 0.7848101265822784


# RANDOM FOREST WITH PCA

In [12]:
#modelling
from sklearn.ensemble import RandomForestClassifier
rf_pca_default = RandomForestClassifier(random_state=True)
rf_pca_default.fit(X_train_pca,y_train)
#prediction
pred_train = rf_pca_default.predict(X_train_pca)
base_pred = rf_pca_default.predict(X_test_pca)
#evaluation
from sklearn.metrics import accuracy_score
print('train accuracy',accuracy_score(pred_train,y_train))
print('test accuracy',accuracy_score(base_pred,y_test))


train accuracy 1.0
test accuracy 0.7468354430379747


# ADA BOOST WITH PCA

In [13]:
from sklearn.model_selection import GridSearchCV

#model
from sklearn.ensemble import AdaBoostClassifier
estimator_ab = AdaBoostClassifier(random_state= True)

#parameters(which you want to tune and identify the best)
param_grid_ab ={'n_estimators':list(range(1,20))}

#grid search
grid_ab = GridSearchCV(estimator_ab,param_grid_ab,scoring ='accuracy',cv= 5)
grid_ab.fit(X_train,y_train)

# best parameters for adaboost model
grid_ab.best_params_

{'n_estimators': 5}

In [14]:
grid_ab.best_estimator_.feature_importances_

array([0. , 0.4, 0. , 0. , 0. , 0.2, 0.2, 0.2])

In [15]:
s1 = pd.DataFrame(grid_ab.best_estimator_.feature_importances_,
                  columns=['featureimportance'],
                  index = X.columns)
imp_features_list_ab = s1[s1['featureimportance']>0].index.to_list()
imp_features_list_ab
                

['Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age']

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score



# Ensure `X_imp` and `y` have the same number of rows
X_imp = X[imp_features_list_ab]

# Correct order of train_test_split
X_train_ab, X_test_ab, y_train_ab, y_test_ab = train_test_split(X_imp, y, test_size=0.2, random_state=42)

# Initialize and train AdaBoostClassifier
ada = AdaBoostClassifier(n_estimators=9, random_state=42)
ada.fit(X_train_pca, y_train_ab)

# Predictions
ypred_train_ab = ada.predict(X_train_pca)
ypred_test_ab = ada.predict(X_test_pca)

# Train accuracy
print('trainaccuracy:', accuracy_score(y_train_ab, ypred_train_ab))

# Test accuracy
print('testaccuracy:', accuracy_score(y_test_ab, ypred_test_ab))


trainaccuracy: 0.7060702875399361
testaccuracy: 0.6075949367088608
